In [ ]:
#!pip3 install cython

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Let's look at our matrix multiplication again

I know I mentioned before to just use numpy but sometimes you're working with a complex function or something that's not as easily sped up with numpy.

Let's assume we have a special matrix multiplication we can't use a generic function for.

In [ ]:
def my_matMul(X,Y):
    # Get size of matrices
    size = len(X)
    # Make a matrix that size with all 0's
    result = np.zeros_like(X)
    # Loop through rows in X/col in Y
    for i in range(size):
        # Loop through rows in Y/col in X
        for j in range(size):
            # Loop through elements in each row/col and 
            for k in range(size):
                # Fill in the resulting matrix with the value at i,j
                result[i][j] += X[i][k] * Y[k][j]
    return result

In [ ]:
size = 200
A = np.random.rand(size,size)
B = np.random.rand(size,size)
%time x = my_matMul(A,B) 

## Let's try cython

From the cython website:

#### [Cython](https://cython.org) gives you the combined power of Python and C to let you

- write Python code that calls back and forth from and to C or C++ code natively at any point.
- easily tune readable Python code into plain C performance by adding static type declarations, also in Python syntax.
- use combined source code level debugging to find bugs in your Python, Cython and C code.
- interact efficiently with large data sets, e.g. using multi-dimensional NumPy arrays.
- quickly build your applications within the large, mature and widely used CPython ecosystem.
- integrate natively with existing code and data from legacy, low-level or high-performance libraries and applications.

##### Personally I use cython to make libraries which call C/C++/Fortarn code, which is already written and fast, in python which has really nice plotting and fitting methods.

We can use cython in jupyter by loading the extension and using `%%cython` in any cell that we want to cythonize and compile to c style code.

In [ ]:
%load_ext Cython

In [ ]:
%%cython 
import numpy as np # we need to redelare any libraries inside the cython cell for cython to load them

### This is all I added! ^^^^

def my_matMul_cy(X,Y):
    # Get size of matrices
    size = len(X)
    # Make a matrix that size with all 0's
    result = np.zeros_like(X)
    # Loop through rows in X/col in Y
    for i in range(size):
        # Loop through rows in Y/col in X
        for j in range(size):
            # Loop through elements in each row/col and 
            for k in range(size):
                # Fill in the resulting matrix with the value at i,j
                result[i][j] += X[i][k] * Y[k][j]
    return result

In [ ]:
size = 200

A = np.random.rand(size,size)
B = np.random.rand(size,size)
%time x = my_matMul_cy(A,B)

<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
## Let's see if we can make it faster

Cython allows you to give your variables types to help speedup the python code that's already written. You can also use the `-a` option to annotate your cython code which shows how one line of python code get's translated into lines of C code. Even if you don't understand C cython will try to help by marking slower functions in the code as red which you can think how to change them.

We can also turn on and off features of python inside of our code which can help to make it faster. https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives

In [ ]:
%%cython -a
import numpy as np # we need to redelare any libraries inside the cython cell for cython to load them

def my_matMul_cy(X,Y):
    # Get size of matrices
    size = len(X)
    # Make a matrix that size with all 0's
    result = np.zeros_like(X)
    # Loop through rows in X/col in Y
    for i in range(size):
        # Loop through rows in Y/col in X
        for j in range(size):
            # Loop through elements in each row/col and 
            for k in range(size):
                # Fill in the resulting matrix with the value at i,j
                result[i][j] += X[i][k] * Y[k][j]
    return result

In [ ]:
size = 200

A = np.random.rand(size,size)
B = np.random.rand(size,size)
%time x = my_matMul_cy(A,B)


<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
### Final Fast solution

In [ ]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

from cython.view cimport array

@cython.boundscheck(False)
@cython.wraparound(False)
def mat_mul_cy_fast(double [:,:] X, double [:,:] Y):
    cdef int i,j,k = 0
    cdef int size = X.shape[0]
    cdef double [:,:] result = array(shape=(size,size), itemsize=sizeof(double), format="d")
    result[:,:] = 0.0
    for i in range(size):
        for k in range(size):
            for j in range(size):
                result[i][j] += X[i][k] * Y[k][j]
    return np.asarray(result)

In [ ]:
size = 200

A = np.random.rand(size,size)
B = np.random.rand(size,size)
%time x = mat_mul_cy_fast(A,B)

In [ ]:
# Still beat by numpy...
size = 200

A = np.random.rand(size,size)
B = np.random.rand(size,size)
%time x = np.matmul(A,B)